In [1]:
import json
import pandas as pd
from textblob import TextBlob
from textblob_fr import PatternTagger, PatternAnalyzer
from vaderSentiment_fr.vaderSentiment import SentimentIntensityAnalyzer

In [2]:
SIA = SentimentIntensityAnalyzer()

In [3]:
# Extract json into dictionary
def load_json_data(dataset_name):
    return json.load(open('../data/' + dataset_name + '.json'))

In [4]:
def preprocess_tweet(tweet_text):
    return tweet_text

In [5]:
def get_labels_rule_based(tweet_text):
    vader_scores = SIA.polarity_scores(tweet_text)
    if vader_scores['compound'] > 0.05:
        vader_label = 'pos'
    elif vader_scores['compound'] < -0.05:
        vader_label = 'neg'
    else:
        vader_label = 'neu'
    textblob_scores = TextBlob(tweet_text, pos_tagger=PatternTagger(), analyzer=PatternAnalyzer()).sentiment
    if textblob_scores[0] > 0.1:
        textblob_label = 'pos'
    elif textblob_scores[0] < -0.1:
        textblob_label = 'neg'
    else:
        textblob_label = 'neu'
    return (vader_label, textblob_label)

In [6]:
def rule_based_sentiment(dataset_name):
    dataset = load_json_data(dataset_name)
    tweet_texts = []
    vader_labels = []
    textblob_labels = []
    
    for tweet, details in dataset.items():
        if details['tweet_data']:
            processed_text = preprocess_tweet(details['tweet_data']['text'])
        else:
            processed_text = details['previous_processed_text']
        tweet_texts.append(processed_text)
        rule_based_labels = get_labels_rule_based(processed_text)
        vader_labels.append(rule_based_labels[0])
        textblob_labels.append(rule_based_labels[1])
        
    return pd.DataFrame({'Tweet Text': tweet_texts, 'VADER Labels': vader_labels, 'TextBlob Labels': textblob_labels})
            
    

In [7]:
rule_based_sentiment('training')

,Tweet Text,VADER Labels,TextBlob Labels
0,Les femmes avec Zemmour vous saluent 🇲🇫 #recon...,neu,neu
1,"On a suggéré une balle dans la tête, on a souh...",neg,neu
2,📹 Emma a 22 ans et elle étudiante en droit. Po...,pos,pos
3,C'est un comble que les pauvres votent pour Ma...,pos,neu
4,"#Cnews #MacronMcKinsey #MacronDegage \n""POUR A...",neg,neu
...,...,...,...
8586,Nous tous Emmanuel Macron avec vous.\nMarine P...,pos,pos
8587,Dans la vie \nSoit on reste à terre \nSoit on ...,pos,pos
8588,"Vivement la réserve, JPP des élections.",neu,neu
8589,Je hais tout ceux qui vont voter Macron vous ê...,neg,neu
